<a href="https://colab.research.google.com/github/vsidaarth/Transformer-Variants-for-HourvAhead-PV-Forecasting/blob/main/PatchTST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import importlib.util
import sys

# Add PatchTST_supervised to Python path
sys.path.append('/content/drive/MyDrive/PatchTST/PatchTST_supervised')

# Load PatchTST model
patchtst_path = "/content/drive/MyDrive/PatchTST/PatchTST_supervised/models/PatchTST.py"
spec = importlib.util.spec_from_file_location("patchtst_model", patchtst_path)
patchtst_module = importlib.util.module_from_spec(spec)
sys.modules["patchtst_model"] = patchtst_module
spec.loader.exec_module(patchtst_module)

# Access model class
PatchTST = patchtst_module.Model

# Load PatchTST_backbone
backbone_path = "/content/drive/MyDrive/PatchTST/PatchTST_supervised/layers/PatchTST_backbone.py"
spec_backbone = importlib.util.spec_from_file_location("patchtst_backbone", backbone_path)
backbone_module = importlib.util.module_from_spec(spec_backbone)
sys.modules["patchtst_backbone"] = backbone_module
spec_backbone.loader.exec_module(backbone_module)

# Access PatchTST_backbone
PatchTST_backbone = backbone_module.PatchTST_backbone



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import plotly.express as px

In [ ]:
# load the data
df = pd.read_csv('/content/updated_timetable.txt')
df.columns=['datetime','irradiation_forecast','temperature_forecast','irradiation','temperature','power']
tdi = pd.to_datetime(df['datetime'], format='%d-%m-%y %H')
df.set_index(tdi, inplace=True)

# # Define date ranges to remove
# ranges_to_remove = [
#     ('2018-05-01', '2018-05-22'),
#     ('2019-09-09', '2019-09-28'),
#     ('2019-11-29', '2020-01-20'),
#     ('2013-07-23', '2013-08-26')
# ]

# # Create a boolean mask for rows to keep
# mask = pd.Series(True, index=df.index)
# for start, end in ranges_to_remove:
#     mask &= ~((df.index >= start) & (df.index <= end))

# # Apply the mask to the DataFrame
# data = df[mask]
start_date = '2014-01-01'
end_date = '2019-01-31'
# Filter the data for the specified date range
data = df[start_date:end_date]

data = data.drop(columns=['datetime','irradiation_forecast','temperature_forecast'])
data['power'] = data['power']/1000

# data['power'] = data['power']/data['power'].max()*317
# data = data[59905::]

In [ ]:
# # === Descriptive audit for PV dataset (train vs last-12-month test) ===
# import numpy as np
# import pandas as pd

# # Ensure expected columns & index
# assert isinstance(data.index, pd.DatetimeIndex), "data must have a DatetimeIndex"
# expected_cols = ["irradiation", "temperature", "power"]
# missing_cols = [c for c in expected_cols if c not in data.columns]
# assert not missing_cols, f"Missing columns in data: {missing_cols}"

# # Define split (last 12 months from end_date)
# end_date_pd = pd.to_datetime("2019-01-31")
# test_start = end_date_pd - pd.DateOffset(months=12)

# train = data[:test_start]
# test  = data[test_start:]

# def basic_stats(df):
#     out = df.agg(['count','mean','std','min','median','max']).T
#     # % of zeros (useful for PV)
#     out['pct_zero'] = (df.eq(0).sum() / df.shape[0] * 100).values
#     # NaNs
#     out['n_nan'] = df.isna().sum().values
#     return out

# def drift_summary(train_s, test_s, name):
#     mu_tr, mu_te = train_s.mean(), test_s.mean()
#     sd_tr, sd_te = train_s.std(ddof=0), test_s.std(ddof=0)
#     return {
#         "var": name,
#         "mean_train": float(mu_tr), "mean_test": float(mu_te),
#         "mean_shift_%": float((mu_te - mu_tr) / (abs(mu_tr) + 1e-12) * 100),
#         "std_train": float(sd_tr), "std_test": float(sd_te),
#         "std_shift_%": float((sd_te - sd_tr) / (sd_tr + 1e-12) * 100),
#     }

# # Global info
# print("=== Coverage ===")
# print(f"Full range: {data.index.min().date()} → {data.index.max().date()} "
#       f"({data.shape[0]} rows)")
# print(f"Train:      {train.index.min().date()} → {train.index.max().date()} "
#       f"({train.shape[0]} rows)")
# print(f"Test:       {test.index.min().date()}  → {test.index.max().date()}  "
#       f"({test.shape[0]} rows)")
# try:
#     print("Inferred frequency:", pd.infer_freq(data.index))
# except Exception:
#     print("Inferred frequency: <unable to infer>")

# # Summary stats
# print("\n=== Summary statistics (FULL) ===")
# display(basic_stats(data).round(4))
# print("\n=== Summary statistics (TRAIN) ===")
# display(basic_stats(train).round(4))
# print("\n=== Summary statistics (TEST) ===")
# display(basic_stats(test).round(4))

# # Train–Test drift checks
# print("\n=== Train–Test drift (means & stds) ===")
# drift_rows = []
# for col in expected_cols:
#     drift_rows.append(drift_summary(train[col].dropna(), test[col].dropna(), col))
# drift_df = pd.DataFrame(drift_rows)
# display(drift_df.round(3))

# # Monthly and diurnal profiles (power)
# print("\n=== Monthly mean power (kW) ===")
# monthly_power = data['power'].resample('M').mean()
# display(monthly_power.to_frame('mean_power_kW').round(3))

# print("\n=== Diurnal profile (average power by hour-of-day) ===")
# hourly_profile = data['power'].groupby(data.index.hour).mean()
# display(hourly_profile.to_frame('mean_power_kW').round(3))

# # Correlations
# print("\n=== Pearson correlation matrix (FULL) ===")
# corr_full = data[expected_cols].corr()
# display(corr_full.round(3))

# print("\n=== Pearson correlation matrix (TRAIN) ===")
# display(train[expected_cols].corr().round(3))

# print("\n=== Pearson correlation matrix (TEST) ===")
# display(test[expected_cols].corr().round(3))

# # Simple outlier flags on power
# q01, q99 = data['power'].quantile([0.01, 0.99])
# outlier_lo = (data['power'] < q01).mean() * 100
# outlier_hi = (data['power'] > q99).mean() * 100
# print("\n=== Power outlier rates (full) ===")
# print(f"<1st percentile: {outlier_lo:.2f}% | >99th percentile: {outlier_hi:.2f}%")

# # Missingness overview
# print("\n=== Missingness (rows with any NaN) ===")
# print(f"Full: {data.isna().any(axis=1).mean()*100:.2f}% "
#       f"| Train: {train.isna().any(axis=1).mean()*100:.2f}% "
#       f"| Test: {test.isna().any(axis=1).mean()*100:.2f}%")

# # Key facts to help writing
# key = {
#     "test_start": str(test_start.date()),
#     "n_rows_full": int(data.shape[0]),
#     "n_rows_train": int(train.shape[0]),
#     "n_rows_test": int(test.shape[0]),
#     "mean_power_train_kW": float(train['power'].mean()),
#     "mean_power_test_kW": float(test['power'].mean()),
#     "irradiation_power_corr_full": float(corr_full.loc['irradiation','power']),
# }
# print("\n=== Key facts ===")
# print(key)


In [ ]:
# import plotly.graph_objects as go

# # Define split point = last 12 months
# test_start = pd.to_datetime(end_date) - pd.DateOffset(months=12)

# # Split into train and test
# train = data[:test_start]
# test  = data[test_start:]

# # Plot with Plotly
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=train.index, y=train['power'],
#     mode='lines',
#     name='Training Data',
#     line=dict(color='blue')
# ))

# fig.add_trace(go.Scatter(
#     x=test.index, y=test['power'],
#     mode='lines',
#     name='Test Unseen Data (last 12 months)',
#     line=dict(color='orange')
# ))

# fig.update_layout(
#     title="Training vs Test Data",
#     xaxis_title="Date",
#     yaxis_title="Power [kW]",
#     template="plotly_white",
#     font=dict(size=24),
#     legend=dict(
#         orientation="h",            # horizontal
#         yanchor="top", y=-0.25,     # push below plot
#         xanchor="center", x=0.5,    # center it
#         font=dict(size=24)
#     ),
#     xaxis=dict(title_font=dict(size=24), tickfont=dict(size=24)),
#     yaxis=dict(title_font=dict(size=24), tickfont=dict(size=24))
# )

# fig.show()


In [ ]:
load = data.loc[:, 'power']
scaler = MinMaxScaler()
load = scaler.fit_transform(load.values.reshape(-1, 1)).flatten()

irradiation = data.loc[:, 'irradiation']
scaler_w = MinMaxScaler()
irradiation = scaler_w.fit_transform(irradiation.values.reshape(-1, 1)).flatten()

irradiation = data.loc[:, 'irradiation']

load_scaled = pd.DataFrame({
    'power_generation': load,
    'irradiation': irradiation
}, index=data.index)


shift_1 = load_scaled.shift(1,axis = 0)
shift_1.columns = ['power_generation(t-1)', 'irradiation(t-1)']

final_table = pd.concat([load_scaled['power_generation'],shift_1],axis = 1)
final_table = final_table.dropna(axis=0)

# Extracting components
final_table['hour'] = final_table.index.hour
final_table['day_of_week'] = final_table.index.weekday
final_table['month'] = final_table.index.month

# Applying cyclical encoding
final_table['hour_sin'] = np.sin(2 * np.pi * final_table['hour'] / 24)
final_table['hour_cos'] = np.cos(2 * np.pi * final_table['hour'] / 24)
final_table['month_sin'] = np.sin(2 * np.pi * final_table['month'] / 12)
final_table['month_cos'] = np.cos(2 * np.pi * final_table['month'] / 12)

# Optionally drop the original time components if they are no longer needed
final_table.drop(['hour', 'day_of_week', 'month'], axis=1, inplace=True)

# Show the modified DataFrame
final_table.head(-1)


,power_generation,power_generation(t-1),irradiation(t-1),hour_sin,hour_cos,month_sin,month_cos
datetime,,,,,,,
2014-01-01 01:00:00,0.0,0.0,0.496861,0.258819,9.659258e-01,0.5,0.866025
2014-01-01 02:00:00,0.0,0.0,0.379389,0.500000,8.660254e-01,0.5,0.866025
2014-01-01 03:00:00,0.0,0.0,0.257452,0.707107,7.071068e-01,0.5,0.866025
2014-01-01 04:00:00,0.0,0.0,0.209292,0.866025,5.000000e-01,0.5,0.866025
2014-01-01 05:00:00,0.0,0.0,0.019541,0.965926,2.588190e-01,0.5,0.866025
...,...,...,...,...,...,...,...
2019-01-31 18:00:00,0.0,0.0,1.006410,-1.000000,-1.836970e-16,0.5,0.866025
2019-01-31 19:00:00,0.0,0.0,1.037107,-0.965926,2.588190e-01,0.5,0.866025
2019-01-31 20:00:00,0.0,0.0,1.346272,-0.866025,5.000000e-01,0.5,0.866025


In [ ]:
test_set_start_date = final_table.index[-1] - pd.DateOffset(months=12)
train_set = final_table[final_table.index < test_set_start_date]
test_set = final_table[final_table.index >= test_set_start_date]

In [ ]:
# # === Config ===
# TARGET_COL   = 'power_generation'
# FEATURE_COLS = None          # None -> use all columns; or set a list manually
# SEQ_LEN      = 96            # lookback
# PRED_LEN     = 1            # horizon
# BATCH_SIZE   = 256
# EPOCHS       = 10
# LR           = 3e-4
# WEIGHT_DECAY = 1e-4
# DROPOUT      = 0.2
# D_MODEL      = 128
# D_FF         = 256
# E_LAYERS     = 3
# N_HEADS      = 8
# PATCH_LEN    = 16
# STRIDE       = 8
# INDIVIDUAL   = True          # separate head per variable; we will index the target channel for loss
# PATIENCE     = 5             # early stopping
# SEED         = 2025

# import numpy as np, pandas as pd, torch, torch.nn as nn, torch.optim as optim
# from types import SimpleNamespace
# from sklearn.preprocessing import StandardScaler
# from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.manual_seed(SEED); np.random.seed(SEED)

# # --- 0) Train/test split already done by you. We assume `final_table`, `train_set`, `test_set` exist. ---
# assert TARGET_COL in final_table.columns, "TARGET_COL not in final_table!"

# # --- 1) Columns / scalers ---
# if FEATURE_COLS is None:
#     FEATURE_COLS = list(final_table.columns)   # all columns, including target
# enc_in = len(FEATURE_COLS)
# tgt_idx = FEATURE_COLS.index(TARGET_COL)

# x_scaler = StandardScaler()
# y_scaler = StandardScaler()

# # fit on TRAIN only
# X_train_df = train_set[FEATURE_COLS].copy()
# y_train_df = train_set[[TARGET_COL]].copy()

# X_train = x_scaler.fit_transform(X_train_df.values)
# y_train = y_scaler.fit_transform(y_train_df.values).ravel()  # (N,)

# # transform test
# X_test = x_scaler.transform(test_set[FEATURE_COLS].values)
# y_test = y_scaler.transform(test_set[[TARGET_COL]].values).ravel()

# # For windowing we need aligned arrays over time for each split
# def build_xy(X, y, seq_len, pred_len):
#     """
#     X: (T, N_feat), y: (T,), both already scaled
#     Returns:
#       X_seq: (num, seq_len, N_feat)
#       Y_seq: (num, pred_len) target-only
#     """
#     T = len(X)
#     num = T - seq_len - pred_len + 1
#     Xs = np.zeros((num, seq_len, X.shape[1]), dtype=np.float32)
#     Ys = np.zeros((num, pred_len), dtype=np.float32)
#     for i in range(num):
#         Xs[i] = X[i:i+seq_len]
#         Ys[i] = y[i+seq_len : i+seq_len+pred_len]
#     return Xs, Ys

# Xtr, Ytr = build_xy(X_train, y_train, SEQ_LEN, PRED_LEN)
# Xte, Yte = build_xy(X_test,  y_test,  SEQ_LEN, PRED_LEN)

# class TSDS(Dataset):
#     def __init__(self, X, Y):
#         self.X = torch.from_numpy(X)     # (B, L, N)
#         self.Y = torch.from_numpy(Y)     # (B, pred_len)
#     def __len__(self): return self.X.shape[0]
#     def __getitem__(self, i): return self.X[i], self.Y[i]

# dtr = DataLoader(TSDS(Xtr, Ytr), batch_size=BATCH_SIZE,
#                  sampler=RandomSampler(TSDS(Xtr, Ytr)), drop_last=True)
# dte = DataLoader(TSDS(Xte, Yte), batch_size=BATCH_SIZE,
#                  sampler=SequentialSampler(TSDS(Xte, Yte)))

# # --- 2) Build PatchTST model (using your imported Model class) ---
# # The supervised repo expects a "configs" namespace with these fields:
# cfg = SimpleNamespace(
#     task_name='forecasting',
#     output_attention=False,
#     enc_in=enc_in,

#     # >>> add the fields your repo expects
#     fc_dropout=DROPOUT,      # e.g., 0.2
#     head_dropout=DROPOUT,    # 0.0–0.2 is typical
#     head_type='flatten',     # forecasting head
#     c_out=enc_in,            # predict all channels; we'll pick target later
#     revin=True,             # set True if you want RevIN

#     seq_len=SEQ_LEN,
#     pred_len=PRED_LEN,
#     patch_len=PATCH_LEN,
#     stride=STRIDE,
#     d_model=D_MODEL,
#     n_heads=N_HEADS,
#     e_layers=E_LAYERS,
#     d_ff=D_FF,
#     dropout=DROPOUT,
#     activation='gelu',
#     affine=True,
#     individual=INDIVIDUAL,
#     pre_norm=False,
#     padding_patch='end',
#     subtract_last=False,   # if your fork supports last-value normalization
#     decomposition=False,       # if True, also set kernel_size
#     kernel_size=25,        # only used when decompose=True
# )


# model = PatchTST(cfg).to(device)

# # quick shape check to avoid "no core" style mistakes
# with torch.no_grad():
#     _x = torch.randn(2, SEQ_LEN, enc_in, device=device)
#     _y = model(_x)   # expected: (2, PRED_LEN, enc_in) if individual=False; or (2, PRED_LEN, enc_in) for most forks
#     print("Sanity forward:", tuple(_y.shape))

# # --- 3) Train setup ---
# # --- 3) Train setup (no val/scheduler/early stopping) ---
# optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# criterion  = nn.MSELoss()

# for epoch in range(1, EPOCHS+1):
#     model.train()
#     tr_loss = 0.0
#     for xb, yb in dtr:
#         xb = xb.to(device)
#         yb = yb.to(device)                 # (B, pred_len)
#         yp = model(xb)[:, :, tgt_idx]      # (B, pred_len)
#         loss = criterion(yp, yb)

#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()

#         tr_loss += loss.item() * xb.size(0)
#     tr_loss /= len(dtr.dataset)
#     print(f"Epoch {epoch:02d} | train MSE {tr_loss:.5f}")


# # --- 4) Evaluation (MAE/RMSE) in original units on the test set ---
# def inverse_to_raw(y_std_flat):
#     """
#     y_std_flat: flattened array in StandardScaler space (which was applied on top of MinMax space).
#     Returns: flattened array in ORIGINAL units (same units as data['power']).
#     """
#     # 1) undo StandardScaler (back to MinMax space)
#     y_mm = y_scaler.inverse_transform(y_std_flat.reshape(-1, 1)).reshape(-1)
#     # 2) undo MinMax (back to original units)
#     y_raw = scaler.inverse_transform(y_mm.reshape(-1, 1)).reshape(-1)
#     return y_raw

# model.eval()
# preds_scaled = []
# truth_scaled = []

# with torch.no_grad():
#     for xb, yb in dte:
#         xb = xb.to(device)
#         yp = model(xb)[:, :, tgt_idx]  # (B, pred_len)
#         preds_scaled.append(yp.detach().cpu().numpy())
#         truth_scaled.append(yb.numpy())

# preds_scaled = np.concatenate(preds_scaled, axis=0)   # (num, pred_len)
# truth_scaled = np.concatenate(truth_scaled, axis=0)

# # back to ORIGINAL units
# preds_real = inverse_to_raw(preds_scaled.ravel()).reshape(preds_scaled.shape)
# truth_real = inverse_to_raw(truth_scaled.ravel()).reshape(truth_scaled.shape)

# mae = np.mean(np.abs(preds_real - truth_real))
# rmse = np.sqrt(np.mean((preds_real - truth_real)**2))
# mbe = np.mean(preds_real - truth_real)
# print(f"Test MAE (real units):  {mae:.4f}")

# print(f"Test RMSE (real units): {rmse:.4f}")
# print(f"Test MBE (real units): {mbe:.4f}")


In [ ]:
def compute_crps(actual, lower, upper):
    """
    Computes CRPS using a quantile-based approximation.
    """
    if actual < lower:
        return (upper - lower) + ((lower - actual) ** 2) / (upper - lower)
    elif actual > upper:
        return (upper - lower) + ((actual - upper) ** 2) / (upper - lower)
    else:
        return (upper - lower) / 2

def compute_winkler_score(y_true, y_lower, y_upper, alpha=0.1):
    """
    Computes the Winkler Score.

    Args:
        y_true (float): The actual observed value.
        y_lower (float): The predicted lower quantile.
        y_upper (float): The predicted upper quantile.
        alpha (float): The significance level (default 0.1 for a 90% prediction interval).

    Returns:
        float: Winkler score for the given forecast.
    """
    interval_width = y_upper - y_lower

    if y_true < y_lower:
        return interval_width + (2 / alpha) * (y_lower - y_true)
    elif y_true > y_upper:
        return interval_width + (2 / alpha) * (y_true - y_upper)
    else:
        return interval_width

In [ ]:
# =========================
# PatchTST (no extra scalers; invert with your original MinMax "scaler")
# =========================
import os, numpy as np, pandas as pd, torch, torch.nn as nn, torch.optim as optim
from types import SimpleNamespace
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# ---- Assumptions from your pipeline ----
# - final_table is already built and scaled (power_generation in MinMax space using `scaler`)
# - train_set, test_set are already split from final_table
# - PatchTST class is imported as PatchTST
# - `scaler` is the SAME MinMaxScaler used on the raw power series

# ---- Config ----
TARGET_COL   = 'power_generation'   # scaled in [0,1] by your original MinMax "scaler"
FEATURE_COLS = None                 # None -> all columns in final_table
SEQ_LEN      = 48
PRED_LEN     = 1
BATCH_SIZE   = 48
EPOCHS       = 10
LR           = 1e-3
WEIGHT_DECAY = 1e-4
DROPOUT      = 0.1
D_MODEL      = 128
D_FF         = 256
E_LAYERS     = 3
N_HEADS      = 4
PATCH_LEN    = 16
STRIDE       = 8
INDIVIDUAL   = True
SEED         = 2025

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(SEED); np.random.seed(SEED)

assert TARGET_COL in final_table.columns, "TARGET_COL missing in final_table"
if FEATURE_COLS is None:
    FEATURE_COLS = list(final_table.columns)

enc_in  = len(FEATURE_COLS)
tgt_idx = FEATURE_COLS.index(TARGET_COL)

# ---- Use the data AS-IS (already scaled); no new scalers here ----
X_train = train_set[FEATURE_COLS].values.astype(np.float32)
y_train = train_set[[TARGET_COL]].values.astype(np.float32).ravel()
X_test  = test_set[FEATURE_COLS].values.astype(np.float32)
y_test  = test_set[[TARGET_COL]].values.astype(np.float32).ravel()

def build_xy(X, y_scaled, seq_len, pred_len):
    T   = len(X)
    num = T - seq_len - pred_len + 1
    Xs  = np.zeros((num, seq_len, X.shape[1]), dtype=np.float32)
    Ys  = np.zeros((num, pred_len), dtype=np.float32)  # still in scaled space
    for i in range(num):
        Xs[i] = X[i:i+seq_len]
        Ys[i] = y_scaled[i+seq_len : i+seq_len+pred_len]
    return Xs, Ys

Xtr, Ytr = build_xy(X_train, y_train, SEQ_LEN, PRED_LEN)
Xte, Yte = build_xy(X_test,  y_test,  SEQ_LEN, PRED_LEN)

class TSDS(Dataset):
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X)   # (B, L, N)
        self.Y = torch.from_numpy(Y)   # (B, pred_len) in scaled space
    def __len__(self):  return self.X.shape[0]
    def __getitem__(self, i): return self.X[i], self.Y[i]

dtr = DataLoader(TSDS(Xtr, Ytr), batch_size=BATCH_SIZE,
                 sampler=RandomSampler(TSDS(Xtr, Ytr)), drop_last=True)
dte = DataLoader(TSDS(Xte, Yte), batch_size=BATCH_SIZE,
                 sampler=SequentialSampler(TSDS(Xte, Yte)))

# ---- Build PatchTST (no RevIN, to avoid hidden normalization) ----
cfg = SimpleNamespace(
    task_name='forecasting', output_attention=False,
    enc_in=enc_in, c_out=enc_in, individual=INDIVIDUAL,
    seq_len=SEQ_LEN, pred_len=PRED_LEN,
    patch_len=PATCH_LEN, stride=STRIDE,
    d_model=D_MODEL, n_heads=N_HEADS, e_layers=E_LAYERS, d_ff=D_FF,
    dropout=DROPOUT, activation='gelu',
    revin=False, affine = True, fc_dropout=DROPOUT, head_dropout=DROPOUT, head_type='flatten',
    pre_norm=False, padding_patch='end', subtract_last=False,
    decomposition=False, kernel_size=25
)
model = PatchTST(cfg).to(device)

# quick shape check
with torch.no_grad():
    _y = model(torch.randn(2, SEQ_LEN, enc_in, device=device))
    print("Forward shape:", tuple(_y.shape))  # usually (2, pred_len, enc_in)

# ---- Train ----
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion  = nn.MSELoss()
for ep in range(1, EPOCHS+1):
    model.train(); tr_loss = 0.0
    for xb, yb in dtr:
        xb = xb.to(device); yb = yb.to(device)             # yb in scaled space
        yp = model(xb)[:, :, tgt_idx]                      # (B, pred_len), scaled
        loss = criterion(yp, yb)
        optimizer.zero_grad(); loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        tr_loss += loss.item() * xb.size(0)
    tr_loss /= len(dtr.dataset)
    print(f"Epoch {ep:02d} | train MSE {tr_loss:.5f}")

# ---- Predict (still scaled), then invert ONCE using your original MinMax "scaler" ----
model.eval()
preds_scaled = []
with torch.no_grad():
    for xb, _ in dte:
        xb = xb.to(device)
        yp = model(xb)[:, :, tgt_idx]                      # scaled
        preds_scaled.append(yp.detach().cpu().numpy())
preds_scaled = np.concatenate(preds_scaled, axis=0)        # (num, pred_len)

# Invert to kW using *your* original MinMax scaler
preds_raw  = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).reshape(preds_scaled.shape)
truth_raw  = scaler.inverse_transform(Yte.reshape(-1, 1)).reshape(Yte.shape)

# Optional: snap tiny residuals to zero (to avoid 3e-08 at night)
preds_raw[np.isclose(preds_raw, 0, atol=1e-6)] = 0.0
truth_raw[np.isclose(truth_raw, 0, atol=1e-12)] = 0.0

# ---- Metrics in kW ----
mae  = float(np.mean(np.abs(preds_raw - truth_raw)))
rmse = float(np.sqrt(np.mean((preds_raw - truth_raw)**2)))
mbe  = float(np.mean(preds_raw - truth_raw))
print(f"Test MAE:  {mae:.4f} kW")
print(f"Test RMSE: {rmse:.4f} kW")
print(f"Test MBE:  {mbe:.4f} kW")

# Flatten to 1D arrays
yhat_act = preds_raw
y_check_act = truth_raw

# # === Correction Rules ===
# # Rule 1: clip negative forecasts
# yhat_act[yhat_act < 0] = 0.0

# # Rule 2: if actual[t] == 0 → forecast[t+1] = 0
# zero_idx = np.where(y_check_act == 0)[0] + 1
# zero_idx = zero_idx[zero_idx < len(yhat_act)]
# yhat_act[zero_idx] = 0.0


# === ACI evaluation after point prediction with model_stacked_mamba ===
y_lowers_mamba_ACI = np.empty(len(yhat_act))
y_uppers_mamba_ACI = np.empty(len(yhat_act))
err_i_mamba_ACI = np.empty(len(yhat_act))
alpha_t_i_mamba_ACI = np.empty(len(yhat_act))
window_i_mamba_ACI = np.empty(len(yhat_act))
res_cal_acp_mamba_ACI = np.empty(len(yhat_act))
alpha_t_mamba_ACI = 0.1
gamma_mamba_ACI = 0.01

# Initialize Metrics
crps_mamba_ACI = np.zeros(len(yhat_act))
winkler_mamba_ACI = np.zeros(len(yhat_act))

for i in range(1, len(yhat_act)):
    res_cal_acp_mamba_ACI = np.abs(yhat_act - y_check_act)

    if alpha_t_mamba_ACI >= 1:
        y_lowers_mamba_ACI[i], y_uppers_mamba_ACI[i] = 0, 0
        err_mamba_ACI = 1
    elif alpha_t_mamba_ACI <= 0:
        y_lowers_mamba_ACI[i], y_uppers_mamba_ACI[i] = 0, 0
        err_mamba_ACI = 0
        alpha_t_mamba_ACI = 0.1
    else:
        window_mamba_ACI = np.quantile(res_cal_acp_mamba_ACI, (1 - alpha_t_mamba_ACI))
        y_lowers_mamba_ACI[i] = yhat_act[i] - window_mamba_ACI
        y_uppers_mamba_ACI[i] = yhat_act[i] + window_mamba_ACI
        err_mamba_ACI = 1 - float((y_lowers_mamba_ACI[i] <= y_check_act[i]) and (y_check_act[i] <= y_uppers_mamba_ACI[i]))

    # Update alpha
    alpha_t_mamba_ACI += gamma_mamba_ACI * (alpha_t_mamba_ACI - err_mamba_ACI)
    if alpha_t_mamba_ACI < 0.1 or alpha_t_mamba_ACI > 0.3:
        alpha_t_mamba_ACI = 0.1

    # Store metrics
    alpha_t_i_mamba_ACI[i] = alpha_t_mamba_ACI
    err_i_mamba_ACI[i] = err_mamba_ACI
    window_i_mamba_ACI[i] = window_mamba_ACI

    crps_mamba_ACI[i] = compute_crps(y_check_act[i], y_lowers_mamba_ACI[i], y_uppers_mamba_ACI[i])
    winkler_mamba_ACI[i] = compute_winkler_score(y_check_act[i], y_lowers_mamba_ACI[i], y_uppers_mamba_ACI[i], alpha=0.1)

# Post-processing: Bound clipping and reshaping
y_lowers_mamba_ACI[y_lowers_mamba_ACI < 0] = 0
y_uppers_mamba_ACI[y_uppers_mamba_ACI > max(y_check_act)] = max(y_check_act)
y_lowers_mamba_ACI = y_lowers_mamba_ACI.reshape(-1, 1)
y_uppers_mamba_ACI = y_uppers_mamba_ACI.reshape(-1, 1)

# Interval metrics
interval_widths_mamba_ACI = y_uppers_mamba_ACI - y_lowers_mamba_ACI
mean_interval_width_mamba_ACI = np.mean(interval_widths_mamba_ACI)
covered_mamba_ACI = (y_check_act >= y_lowers_mamba_ACI) & (y_check_act <= y_uppers_mamba_ACI)
coverage_percentage_mamba_ACI = np.mean(covered_mamba_ACI) * 100
mean_crps_mamba_ACI = np.mean(crps_mamba_ACI)
mean_winkler_mamba_ACI = np.mean(winkler_mamba_ACI)

# Final results
print(f"Mean Interval Width: {mean_interval_width_mamba_ACI}")
print(f"Coverage Percentage: {coverage_percentage_mamba_ACI}%")
print(f"Mean CRPS: {mean_crps_mamba_ACI}")
print(f"Mean Winkler Score: {mean_winkler_mamba_ACI}")



Forward shape: (2, 1, 7)
Epoch 01 | train MSE 0.03074
Epoch 02 | train MSE 0.01635
Epoch 03 | train MSE 0.01564
Epoch 04 | train MSE 0.01549
Epoch 05 | train MSE 0.01432
Epoch 06 | train MSE 0.01334
Epoch 07 | train MSE 0.01391
Epoch 08 | train MSE 0.01490
Epoch 09 | train MSE 0.01355
Epoch 10 | train MSE 0.01277
Test MAE:  0.1912 kW
Test RMSE: 0.3501 kW
Test MBE:  -0.0471 kW


/tmp/ipython-input-557394183.py:175: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/tmp/ipython-input-557394183.py:176: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/tmp/ipython-input-557394183.py:177: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/tmp/ipython-input-557394183.py:189: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing th

Mean Interval Width: 0.6237166094736186
Coverage Percentage: 86.17009066911511%
Mean CRPS: 0.5387949939319495
Mean Winkler Score: 1.8610488975033919


In [ ]:
# import itertools, random, time
# import numpy as np, pandas as pd, torch
# import torch.nn as nn, torch.optim as optim
# from types import SimpleNamespace
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# # === ACI function ===
# def run_ACI(yhat_act, y_check_act, alpha0=0.1, gamma=0.01):
#     n = len(yhat_act)
#     y_lowers = np.empty(n)
#     y_uppers = np.empty(n)
#     crps = np.zeros(n)
#     winkler = np.zeros(n)
#     alpha_t = alpha0

#     for i in range(1, n):
#         res_cal = np.abs(yhat_act - y_check_act)
#         if alpha_t >= 1:
#             y_lowers[i], y_uppers[i] = 0, 0
#             err = 1
#         elif alpha_t <= 0:
#             y_lowers[i], y_uppers[i] = 0, 0
#             err = 0
#             alpha_t = alpha0
#         else:
#             window = np.quantile(res_cal, (1 - alpha_t))
#             y_lowers[i] = yhat_act[i] - window
#             y_uppers[i] = yhat_act[i] + window
#             err = 1 - float((y_lowers[i] <= y_check_act[i]) and (y_check_act[i] <= y_uppers[i]))
#         # update alpha
#         alpha_t += gamma * (alpha_t - err)
#         if alpha_t < 0.1 or alpha_t > 0.3:
#             alpha_t = alpha0
#         # metrics
#         crps[i] = compute_crps(y_check_act[i], y_lowers[i], y_uppers[i])
#         winkler[i] = compute_winkler_score(y_check_act[i], y_lowers[i], y_uppers[i], alpha=alpha0)

#     # clip intervals
#     y_lowers[y_lowers < 0] = 0
#     y_uppers[y_uppers > max(y_check_act)] = max(y_check_act)

#     # summary metrics
#     interval_widths = y_uppers - y_lowers
#     mean_interval_width = np.mean(interval_widths)
#     covered = (y_check_act >= y_lowers) & (y_check_act <= y_uppers)
#     coverage = np.mean(covered) * 100
#     mean_crps = np.mean(crps)
#     mean_winkler = np.mean(winkler)

#     return coverage, mean_interval_width, mean_crps, mean_winkler


# # --- Define search space ---
# param_grid = {
#     "SEQ_LEN":   [24],
#     "BATCH_SIZE": [48],
#     "LR":        [0.001],
#     "D_MODEL":   [64],
#     "N_HEADS":   [4],
#     "PATCH_LEN": [8],
#     "STRIDE":    [8],
#     "E_LAYERS":  [3],
# }

# # Generate all combinations (or sample a subset)
# all_combos = list(itertools.product(*param_grid.values()))
# print(f"Total combinations: {len(all_combos)}")
# random.shuffle(all_combos)
# all_combos = all_combos[:20]   # <-- sample 20 for speed, adjust as needed


# # --- Utility: train + eval PatchTST once ---
# def train_eval_patchtst(params, train_set, test_set, scaler, TARGET_COL="power_generation"):
#     SEQ_LEN, BATCH_SIZE, LR, D_MODEL, N_HEADS, PATCH_LEN, STRIDE, E_LAYERS = params

#     # === Prepare data ===
#     FEATURE_COLS = list(train_set.columns)
#     tgt_idx = FEATURE_COLS.index(TARGET_COL)

#     def build_xy(X, y, seq_len, pred_len):
#         T = len(X); num = T - seq_len - pred_len + 1
#         Xs = np.zeros((num, seq_len, X.shape[1]), dtype=np.float32)
#         Ys = np.zeros((num, pred_len), dtype=np.float32)
#         for i in range(num):
#             Xs[i] = X[i:i+seq_len]
#             Ys[i] = y[i+seq_len : i+seq_len+pred_len]
#         return Xs, Ys

#     Xtr, Ytr = build_xy(
#         train_set[FEATURE_COLS].values.astype(np.float32),
#         train_set[[TARGET_COL]].values.astype(np.float32).ravel(),
#         SEQ_LEN, 1
#     )
#     Xte, Yte = build_xy(
#         test_set[FEATURE_COLS].values.astype(np.float32),
#         test_set[[TARGET_COL]].values.astype(np.float32).ravel(),
#         SEQ_LEN, 1
#     )

#     # Torch datasets
#     class TSDS(torch.utils.data.Dataset):
#         def __init__(self, X, Y):
#             self.X = torch.from_numpy(X); self.Y = torch.from_numpy(Y)
#         def __len__(self): return self.X.shape[0]
#         def __getitem__(self,i): return self.X[i], self.Y[i]

#     dtr = DataLoader(TSDS(Xtr, Ytr), batch_size=BATCH_SIZE,
#                      sampler=RandomSampler(TSDS(Xtr, Ytr)), drop_last=True)
#     dte = DataLoader(TSDS(Xte, Yte), batch_size=BATCH_SIZE,
#                      sampler=SequentialSampler(TSDS(Xte, Yte)))

#     # === Build model ===
#     cfg = SimpleNamespace(
#         task_name='forecasting', output_attention=False,
#         enc_in=len(FEATURE_COLS), c_out=len(FEATURE_COLS), individual=True,
#         seq_len=SEQ_LEN, pred_len=1,
#         patch_len=PATCH_LEN, stride=STRIDE,
#         d_model=D_MODEL, n_heads=N_HEADS, e_layers=E_LAYERS, d_ff=256,
#         dropout=0.1, activation='gelu',
#         revin=False, fc_dropout=0.1, head_dropout=0.1, head_type='flatten',
#         pre_norm=False, padding_patch='end', subtract_last=False,
#         decomposition=False, kernel_size=25, affine=True
#     )
#     model = PatchTST(cfg).to(device)

#     optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
#     criterion = nn.MSELoss()

#     # === Train quick ===
#     for ep in range(5):   # short run for search
#         model.train(); tr_loss = 0
#         for xb, yb in dtr:
#             xb, yb = xb.to(device), yb.to(device)
#             yp = model(xb)[:, :, tgt_idx]
#             loss = criterion(yp, yb)
#             optimizer.zero_grad(); loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#             optimizer.step()
#             tr_loss += loss.item() * xb.size(0)
#         tr_loss /= len(dtr.dataset)

#     # === Evaluate point metrics ===
#     model.eval(); preds_scaled = []
#     with torch.no_grad():
#         for xb, yb in dte:
#             xb = xb.to(device)
#             yp = model(xb)[:, :, tgt_idx]
#             preds_scaled.append(yp.detach().cpu().numpy())
#     preds_scaled = np.concatenate(preds_scaled, axis=0)

#     preds_raw = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).ravel()
#     truth_raw = scaler.inverse_transform(Yte.reshape(-1, 1)).ravel()

#     mae = np.mean(np.abs(preds_raw - truth_raw))
#     rmse = np.sqrt(np.mean((preds_raw - truth_raw)**2))
#     mbe = np.mean(preds_raw - truth_raw)

#     # === ACI metrics ===
#     coverage, miw, crps, winkler = run_ACI(preds_raw, truth_raw)

#     return mae, rmse, mbe, coverage, miw, crps, winkler


# # --- Run search ---
# results = []
# for combo in all_combos:
#     try:
#         mae, rmse, mbe, coverage, miw, crps, winkler = train_eval_patchtst(combo, train_set, test_set, scaler)
#         results.append({
#             "SEQ_LEN": combo[0], "BATCH_SIZE": combo[1], "LR": combo[2],
#             "D_MODEL": combo[3], "N_HEADS": combo[4], "PATCH_LEN": combo[5],
#             "STRIDE": combo[6], "E_LAYERS": combo[7],
#             "MAE": mae, "RMSE": rmse, "MBE": mbe,
#             "Coverage%": coverage, "MIW": miw, "CRPS": crps, "Winkler": winkler
#         })
#         print(f"Done {combo} → RMSE={rmse:.3f}, Coverage={coverage:.1f}%")
#     except Exception as e:
#         print("Failed:", combo, "error:", str(e))

# df_search = pd.DataFrame(results).sort_values("Coverage%", ascending=False)
# print(df_search.head(10))


In [ ]:
 # import os
# import pandas as pd
# import numpy as np

# # === Function to save forecasts ===
# def save_forecasts(truth_orig, preds_orig, test_set, model_name, filename):
#     # Ensure save folder exists in Google Drive
#     save_dir = "/content/drive/MyDrive/Colab Notebooks/Forecasts"
#     os.makedirs(save_dir, exist_ok=True)

#     # Build full output path
#     out_path = os.path.join(save_dir, filename)

#     # Align lengths
#     n = min(len(truth_orig), len(preds_orig), len(test_set.index))

#     # Build dataframe
#     df = pd.DataFrame({
#         "datetime": pd.to_datetime(test_set.index[:n]),
#         "actual": np.asarray(truth_orig).reshape(-1)[:n],
#         "forecast": np.asarray(preds_orig).reshape(-1)[:n],
#         "model": model_name
#     })

#     # Save to CSV
#     df.to_csv(out_path, index=False)
#     print(f"[{model_name}] CSV saved to: {out_path}")
#     return df.head()

# # === Example: Save Informer results ===
# save_forecasts(
#     truth_raw,      # your ground truth (kW)
#     preds_raw,      # your model forecasts (kW)
#     test_set,                  # DataFrame with datetime index
#     "PatchTST",                # model name
#     "preds_PatchTST2.csv"       # file name in Drive
# )


In [ ]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.metrics import mean_absolute_error, mean_squared_error


# results = []      # <--- reset before the loop
# corrected = {}    # <--- reset too, if you’re saving corrected DataFrames

# # --- Paths ---
# base = "/content/drive/MyDrive/Colab Notebooks/Forecasts"
# files = { "Autoformer": os.path.join(base, "preds_Autoformer.csv"),
#          "Informer": os.path.join(base, "preds_informer.csv"),
#           "FEDformer": os.path.join(base, "preds_FEDformer.csv"),
#           "PatchTST": os.path.join(base, "preds_PatchTST2.csv"),
#           "DLinear": os.path.join(base, "preds_linear.csv"), }

# for name, path in files.items():
#     df = pd.read_csv(path, parse_dates=["datetime"])
#     df = df.sort_values("datetime").reset_index(drop=True)

#     # Rule 1: if actual[t] == 0 → forecast[t+1] = 0
#     zero_idx = df.index[df["actual"] == 0] + 1
#     zero_idx = zero_idx[zero_idx < len(df)]
#     df.loc[zero_idx, "forecast"] = 0.0

#     # Rule 2: clip negatives
#     df["forecast"] = df["forecast"].clip(lower=0)

#     # Store corrected DF
#     corrected[name] = df.copy()

#     # Metrics
#     mae  = mean_absolute_error(df["actual"], df["forecast"])
#     rmse = np.sqrt(mean_squared_error(df["actual"], df["forecast"]))
#     mbe  = (df["forecast"] - df["actual"]).mean()

#     results.append({
#         "Model": name,
#         "MAE (kW)": mae,
#         "RMSE (kW)": rmse,
#         "MBE (kW)": mbe,
#         "MAE (% of 5kW peak)": mae/5*100,
#         "RMSE (% of 5kW peak)": rmse/5*100,
#         "MBE (% of 5kW peak)": mbe/5*100,
#     })

# metrics_df = pd.DataFrame(results)
# print(metrics_df.round(4))


In [ ]:
# # === Extract PatchTST series for ACI ===
# df_patch = corrected["PatchTST"].copy()


# # These are the actuals and forecasts after your rules (zero clipping etc.)
# y_check_act = df_patch["actual"].values    # ground truth
# yhat_act    = df_patch["forecast"].values  # point forecasts

In [ ]:
print(test_set.shape, test_set[1:2].index, test_set[-1:].index)

(8761, 7) DatetimeIndex(['2018-02-01'], dtype='datetime64[ns]', name='datetime', freq=None) DatetimeIndex(['2019-01-31 23:00:00'], dtype='datetime64[ns]', name='datetime', freq=None)


In [ ]:
y_check_act_ACI = np.squeeze(y_check_act)
y_uppers_ACI = np.squeeze(y_uppers_mamba_ACI)
y_lowers_ACI = np.squeeze(y_lowers_mamba_ACI)
yhat_act_stacked_ACI = np.squeeze(yhat_act)
y_check_act_ACI = np.squeeze(y_check_act_ACI)
y_uppers_ACI = np.squeeze(y_uppers_ACI)
y_lowers_ACI = np.squeeze(y_lowers_ACI)
yhat_act_stacked_ACI = np.squeeze(yhat_act_stacked_ACI)
acp_ACI = pd.DataFrame([y_check_act_ACI,y_uppers_ACI,y_lowers_ACI]).T
acp_ACI.columns = ['Actual', 'upper', 'lower']
acp_ACI.index = test_set[48::].index

In [ ]:
start_date = pd.to_datetime('2018-02-05 00:00:00')
end_date = start_date + pd.Timedelta(hours=22)
acp1 = acp_ACI[start_date:end_date]


start_date = pd.to_datetime('2018-05-28 00:00:00')
end_date = start_date + pd.Timedelta(hours=22)
acp2 = acp_ACI[start_date:end_date]


start_date = pd.to_datetime('2018-08-02 00:00:00')
end_date = start_date + pd.Timedelta(hours=22)
acp3 = acp_ACI[start_date:end_date]



start_date = pd.to_datetime('2018-12-21 00:00:00')
end_date = start_date + pd.Timedelta(hours=22)
acp4 = acp_ACI[start_date:end_date]

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Four days plot

fig = make_subplots(
    rows=2, cols=2
)

fig.update_xaxes(title_text="Time (h)", row=1, col=1)
fig.update_xaxes(title_text="Time (h)", row=1, col=2)
fig.update_xaxes(title_text="Time (h)", row=2, col=1)
fig.update_xaxes(title_text="Time (h)", row=2, col=2)
fig.update_yaxes(title_text="Power (MW)", row=1, col=1, nticks=6)
fig.update_yaxes(title_text="Power (MW)", row=1, col=2,nticks=6)
fig.update_yaxes(title_text="Power (MW)", row=2, col=1, nticks=6)
fig.update_yaxes(title_text="Power (MW)", row=2, col=2,nticks=6)

# Updating the font size of x-axis and y-axis titles
fig.update_xaxes(title_font=dict(size=24), row='all', col='all')
fig.update_yaxes(title_font=dict(size=24), row='all', col='all')

# Updating the font size of the axis tick labels
fig.update_xaxes(tickfont=dict(size=24), row='all', col='all')
fig.update_yaxes(tickfont=dict(size=24), row='all', col='all')

# Updating the layout for the legend, title, and overall font
fig.update_layout(
    legend=dict(font=dict(size=24)),
    title=dict(font=dict(size=24)),
    font=dict(size=24)
)


fig.append_trace(go.Scatter(x= acp1.index.to_pydatetime(), y = acp1['Actual'],
    name = 'Actual', legendgroup = '1', line=dict(color='blue', dash='dash'),
    mode='lines+markers'), row=1, col=1)

fig.append_trace(go.Scatter(name='PatchTST ACI', x=acp1.index, y=acp1['upper'],
    mode='lines+markers', marker=dict(symbol='star'), line=dict(color='red', width=1),
    legendgroup = '2'), row=1, col=1)

fig.append_trace(go.Scatter(name='Lower Bound ACI', x=acp1.index, y=acp1['lower'],
    line=dict(color='red', width=1), mode='lines+markers', showlegend=False,
    marker=dict(symbol='star'), legendgroup = '2'), row=1, col=1)

# Second subplot - hide duplicate legend entries
fig.append_trace(go.Scatter(x= acp2.index.to_pydatetime(), y = acp2['Actual'],
    name = 'Actual', legendgroup = '1', line=dict(color='blue', dash='dash'),
    mode='lines+markers', showlegend=False), row=1, col=2)

fig.append_trace(go.Scatter(name='PatchTST ACI', x=acp2.index, y=acp2['upper'],
    mode='lines+markers', marker=dict(symbol='star'), line=dict(color='red', width=1),
    legendgroup = '2', showlegend=False), row=1, col=2)

fig.append_trace(go.Scatter(name='Lower Bound ACI', x=acp2.index, y=acp2['lower'],
    line=dict(color='red', width=1), mode='lines+markers', showlegend=False,
    marker=dict(symbol='star'), legendgroup = '2'), row=1, col=2)

# Third subplot - hide duplicate legend entries
fig.append_trace(go.Scatter(x= acp3.index.to_pydatetime(), y = acp3['Actual'],
    name = 'Actual', legendgroup = '1', line=dict(color='blue', dash='dash'),
    mode='lines+markers', showlegend=False), row=2, col=1)

fig.append_trace(go.Scatter(name='PatchTST ACI', x=acp3.index, y=acp3['upper'],
    mode='lines+markers', marker=dict(symbol='star'), line=dict(color='red', width=1),
    legendgroup = '2', showlegend=False), row=2, col=1)

fig.append_trace(go.Scatter(name='Lower Bound ACI', x=acp3.index, y=acp3['lower'],
    line=dict(color='red', width=1), mode='lines+markers', showlegend=False,
    marker=dict(symbol='star'), legendgroup = '2'), row=2, col=1)

# Fourth subplot - hide duplicate legend entries
fig.append_trace(go.Scatter(x= acp4.index.to_pydatetime(), y = acp4['Actual'],
    name = 'Actual', legendgroup = '1', line=dict(color='blue', dash='dash'),
    mode='lines+markers', showlegend=False), row=2, col=2)

fig.append_trace(go.Scatter(name='PatchTST ACI', x=acp4.index, y=acp4['upper'],
    mode='lines+markers', marker=dict(symbol='star'), line=dict(color='red', width=1),
    legendgroup = '2', showlegend=False), row=2, col=2)

fig.append_trace(go.Scatter(name='Lower Bound ACI', x=acp4.index, y=acp4['lower'],
    line=dict(color='red', width=1), mode='lines+markers', showlegend=False,
    marker=dict(symbol='star'), legendgroup = '2'), row=2, col=2)





for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 24

fig.update_layout(
    height=1200,
    width=2000,  # Consider increasing this if it's feasible
    template='simple_white',
    title={
        'text': "Interval Forecast Performance",
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(size=20, color="Black"),
    title_font=dict(size=24, color="Black"),
    legend=dict(
        orientation="h",  # Horizontal layout
        yanchor="bottom",  # Anchored at the bottom
        y=-0.2,  # Adjust this value to move the legend down
        xanchor="center",  # Center anchor
        x=0.5  # Centered horizontally
    )
)

fig.show()